# Set references for v2_el tuning and export to ONNX


In [1]:
from core import crossval_table, fit_table
from Gaugi import mkdir_p
from sklearn.model_selection import StratifiedKFold, KFold
from multiprocessing import Process
from time import sleep
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import json
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, model_from_json


from pprint import pprint
from copy import deepcopy, copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...


In [2]:
output_path = 'output/fitting'
mkdir_p(output_path)

In [13]:
def create_op_dict(op, decoration='reference'):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : decoration+"/"+op+"/pd_ref#0",
              'fa_ref'    : decoration+"/"+op+"/fa_ref#0",
              'sp_ref'    : decoration+"/"+op+"/sp_ref",
              'pd_val'    : decoration+"/"+op+"/pd_val#0",
              'fa_val'    : decoration+"/"+op+"/fa_val#0",
              'sp_val'    : decoration+"/"+op+"/sp_val",
              'pd_op'     : decoration+"/"+op+"/pd_op#0",
              'fa_op'     : decoration+"/"+op+"/fa_op#0",
              'sp_op'     : decoration+"/"+op+"/sp_op",

              # Counts
              'pd_ref_passed'    : decoration+"/"+op+"/pd_ref#1",
              'fa_ref_passed'    : decoration+"/"+op+"/fa_ref#1",
              'pd_ref_total'     : decoration+"/"+op+"/pd_ref#2",
              'fa_ref_total'     : decoration+"/"+op+"/fa_ref#2",
              'pd_val_passed'    : decoration+"/"+op+"/pd_val#1",
              'fa_val_passed'    : decoration+"/"+op+"/fa_val#1",
              'pd_val_total'     : decoration+"/"+op+"/pd_val#2",
              'fa_val_total'     : decoration+"/"+op+"/fa_val#2",
              'pd_op_passed'     : decoration+"/"+op+"/pd_op#1",
              'fa_op_passed'     : decoration+"/"+op+"/fa_op#1",
              'pd_op_total'      : decoration+"/"+op+"/pd_op#2",
              'fa_op_total'      : decoration+"/"+op+"/fa_op#2",
    })
    return d


op_names = ['loosetrk']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op, "reference")

In [14]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [15]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [16]:
cv.from_csv('output/crossval/table_v2_el.csv')

In [17]:
best_inits = cv.filter_inits("max_sp_val")
best_inits = best_inits.loc[(best_inits.model_idx==0)] # 5 neurons for all phase spaces
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')

In [18]:
basepath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
datapath = basepath+'/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(5)]

In [19]:
# calculate all pd/fa from reference file
ref_path = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins/'
ref_path += 'new_references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.ref.npz'
ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_values = [[ {} for eta in range(5)] for et in range(5)]

from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for op_name in op_names:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            _pd = refObj.getSgnPassed(op_name)/refObj.getSgnTotal(op_name)
            _fa = refObj.getBkgPassed(op_name)/refObj.getBkgTotal(op_name)
            ref_values[et_bin][eta_bin][op_name] = {'pd':_pd, 'fa':_fa, 'pd_epsilon':0.0}

In [20]:
def getPatterns( path, etbin, etabin ):

    pidname = 'el_lhmedium'
    from kepler.pandas import load_hdf
    import numpy as np
    df = load_hdf(path)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | ((df.target==0) & (df['el_lhvloose']==False) ) ]
    df = df.loc[ (df['trig_L2_cl_et']/1000  >= etbin[0]) & (df['trig_L2_cl_et']/1000  < etbin[1])]
    df = df.loc[ (abs(df['trig_L2_cl_eta'])  >= etabin[0]) & (abs(df['trig_L2_cl_eta'])  < etabin[1])]

    df = df.loc[ df['trig_L2_el_hastrack'] == True ] # only rows with track information 


    # for new training, we selected 1/2 of rings in each layer
    #pre-sample - 8 rings
    # EM1 - 64 rings
    # EM2 - 8 rings
    # EM3 - 8 rings
    # Had1 - 4 rings
    # Had2 - 4 rings
    # Had3 - 4 rings
    prefix = 'trig_L2_cl_ring_%i'

    # rings presmaple 
    presample = [prefix %iring for iring in range(8//2)]
    # EM1 list
    sum_rings = 8
    em1 = [prefix %iring for iring in range(sum_rings, sum_rings+(64//2))]
    # EM2 list
    sum_rings = 8+64
    em2 = [prefix %iring for iring in range(sum_rings, sum_rings+(8//2))]
    # EM3 list
    sum_rings = 8+64+8
    em3 = [prefix %iring for iring in range(sum_rings, sum_rings+(8//2))]
    # HAD1 list
    sum_rings = 8+64+8+8
    had1 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    # HAD2 list
    sum_rings = 8+64+8+8+4
    had2 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    # HAD3 list
    sum_rings = 8+64+8+8+4+4
    had3 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    col_names = presample+em1+em2+em3+had1+had2+had3
    rings = df[col_names].values.astype(np.float32)

    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    avgmu = df.avgmu.values
    target = df['target'].values.astype(np.int16)
    data_rings = norm1(rings)
    
    n = data_rings.shape[0]
    data_etOverPt = df['trig_L2_el_etOverPt'].astype(np.float32).to_numpy().reshape((n,1))  / 1
    data_deta     = df['trig_L2_el_trkClusDeta'].astype(np.float32).to_numpy().reshape((n,1))  / 1.
    data_dphi     = df['trig_L2_el_trkClusDphi'].astype(np.float32).to_numpy().reshape((n,1))  / 1.
    

    data_trk    = np.concatenate( (data_etOverPt, data_deta, data_dphi), axis=1)
 
    return [data_rings, data_trk], target, avgmu



## Linear correction:

In [21]:
kf = StratifiedKFold(n_splits=10, random_state=512, shuffle=True)

In [22]:
cv_fit = fit_table( etbins, etabins, kf )

**NOTE**: This step will take something like 5 min to be ready

In [23]:
best_sorts_refit = cv_fit.fill( best_sorts , getPatterns, paths, ref_values, output_path=output_path ,
                                min_avgmu=16, max_avgmu=60)

Filling... : 100%|████████████████████| 25/25 [07:58<00:00, 19.15s/it]


In [24]:
best_sorts_refit.to_csv('output/fitting/best_sorts_v2_el.csv')

In [25]:
for op in op_names:
    cv_fit.dump_beamer_table( best_sorts_refit.loc[best_sorts_refit.op_name == op] ,                  
                              op+' Fitting (v2-el)', 'fitting_v2_el_'+op)

2022-03-28 18:53:05,407 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v2_el_loosetrk.pdf latex code...


In [26]:
!mv *.pdf output/fitting
!mv *.tex output/fitting

## Export all tunings:

In [29]:
model_name_format = 'data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v2_el.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
op_capnames = ['Loose']
for idx, op in enumerate( ['loosetrk'] ):
    cv_fit.export(best_sorts_refit.loc[best_sorts_refit.op_name==op], 
                  model_name_format.format(op=op_capnames[idx]), 
                  config_name_format.format(op=op_capnames[idx]), 
                  op, 
                  to_onnx     = True,
                  remove_last = True,
                  min_avgmu   = 16,
                  max_avgmu   = 100)

2022-03-28 18:59:37,295 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerLooseTriggerConfig.conf.


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -

In [30]:
!mv models output/fitting
!mv *.conf output/fitting
!mv *.tex output/fitting
!mv *.pdf output/fitting

mv: cannot stat '*.tex': No such file or directory
mv: cannot stat '*.pdf': No such file or directory
